# Ramsey King
# DSC 550 - Data Mining
# September 9, 2021
# Week 2: Handling Categorical Data, Text, Dates & Times
# Exercise 2.2 


You can find the dataset controversial-comments.jsonl for this exercise in the Weekly Resources: Week 2 Data Files.
Pre-processing Text: For this part, you will start by reading the controversial-comments.jsonl file into a DataFrame.

In [1]:
import pandas as pd
import numpy as np

comments_df = pd.read_json('controversial-comments.jsonl', lines=True)
comments_df.head()
# I am going to limit the number of entries from 950,000 to 50,000 as per the instructions in the assignment to help with the runtime.

# comments_df = comments_df[:50000].copy()
comments_df = comments_df.sample(50000).copy()


A. Convert all text to lowercase letters.

In [2]:
comments_df['txt'] = comments_df['txt'].apply(str.lower)
comments_df['txt'][:5]

707384    https://mobile.twitter.com/realdonaldtrump/sta...
169379    tillerson, carson, devos, puzder, pruitt...\n\...
311623                                 *crickets, crickets*
58296     he's saying there are varying levels of bad. g...
533523     so what i'm hearing you say is vote third party.
Name: txt, dtype: object

B. Remove all punctuation from the text.


In [3]:
import re
comments_df['txt'] = comments_df['txt'].apply(lambda remove_punct: re.sub(r'[^\w\s]', '', remove_punct))
comments_df['txt'][:5]

707384    httpsmobiletwittercomrealdonaldtrumpstatus3319...
169379    tillerson carson devos puzder pruitt\n\nplease...
311623                                    crickets crickets
58296     hes saying there are varying levels of bad get...
533523       so what im hearing you say is vote third party
Name: txt, dtype: object

C. Remove stop words.


In [4]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# nltk.download('stopwords')
stop_words = stopwords.words('english')

def rem_stop_words(rem_list):
    return [word for word in rem_list if word not in stop_words]

comments_df['txt_tokenized'] = comments_df['txt'].apply(word_tokenize)
comments_df['txt_tokenized'] = comments_df['txt_tokenized'].apply(rem_stop_words)
comments_df['txt_tokenized'][:5]

707384    [httpsmobiletwittercomrealdonaldtrumpstatus331...
169379    [tillerson, carson, devos, puzder, pruitt, ple...
311623                                 [crickets, crickets]
58296     [hes, saying, varying, levels, bad, getting, h...
533523               [im, hearing, say, vote, third, party]
Name: txt_tokenized, dtype: object

D. Apply NLTK’s PorterStemmer.

In [5]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def apply_porter_stemmer(tokenized_words):
    return [porter.stem(word) for word in tokenized_words]

comments_df['txt_tokenized_stemmed'] = comments_df['txt_tokenized'].apply(apply_porter_stemmer)
comments_df['txt_tokenized_stemmed'][:5]

707384    [httpsmobiletwittercomrealdonaldtrumpstatus331...
169379    [tillerson, carson, devo, puzder, pruitt, plea...
311623                                   [cricket, cricket]
58296     [he, say, vari, level, bad, get, hit, car, bad...
533523                  [im, hear, say, vote, third, parti]
Name: txt_tokenized_stemmed, dtype: object

Now that the data is pre-processed, you will apply three different techniques to get it into a usable form for
model-building. Apply each of the following steps (individually) to the pre-processed data.

A. Convert each text entry into a word-count vector (see sects 5.3 & 6.8 in the Machine Learning with Python Cookbook).

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
results = count.fit_transform(comments_df['txt'])
print(comments_df.info())
print(type(results))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 707384 to 500006
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   con                    50000 non-null  int64 
 1   txt                    50000 non-null  object
 2   txt_tokenized          50000 non-null  object
 3   txt_tokenized_stemmed  50000 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.9+ MB
None
<class 'scipy.sparse.csr.csr_matrix'>


B. Convert each text entry into a part-of-speech tag vector (see section 6.7 in the Machine Learning with Python Cookbook).

In [7]:
from nltk import pos_tag
from nltk import word_tokenize
from sklearn.preprocessing import MultiLabelBinarizer
# nltk.download('averaged_perceptron_tagger')
tweets = comments_df['txt'].tolist()
tagged_tweets = []
for tweet in tweets:
    tweet_tag = nltk.pos_tag(word_tokenize(tweet))
    tagged_tweets.append([tag for word, tag in tweet_tag])

one_hot_multi = MultiLabelBinarizer()
one_hot_multi.fit_transform(tagged_tweets)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 1, 0, 1]])

C. Convert each entry into a term frequency-inverse document frequency (tfidf) vector
(see section 6.9 in the Machine Learning with Python Cookbook).

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
text_data = np.array(tweets)
tfidf = TfidfVectorizer()
feature_matrix = tfidf.fit_transform(text_data)
tfidf.vocabulary_

{'httpsmobiletwittercomrealdonaldtrumpstatus331907383771148288ref_srctwsrc5etfw': 19952,
 'tillerson': 41028,
 'carson': 7116,
 'devos': 11339,
 'puzder': 32653,
 'pruitt': 32388,
 'please': 30918,
 'tell': 40284,
 'me': 25916,
 'how': 19392,
 'those': 40850,
 'people': 30282,
 'fit': 15111,
 'the': 40512,
 'position': 31332,
 'they': 40655,
 'were': 44465,
 'appointed': 3692,
 'edit': 12918,
 'fuck': 15878,
 'for': 15445,
 'criticizing': 9964,
 'presidentelect': 31751,
 'who': 44652,
 'refuses': 33676,
 'to': 41147,
 'hold': 19134,
 'press': 31768,
 'conference': 8935,
 'and': 3198,
 'communicates': 8652,
 'with': 44893,
 'america': 3073,
 'via': 43632,
 'twitter': 42064,
 'crickets': 9905,
 'hes': 18808,
 'saying': 35654,
 'there': 40618,
 'are': 3796,
 'varying': 43489,
 'levels': 24424,
 'of': 28755,
 'bad': 4578,
 'getting': 16411,
 'hit': 19054,
 'by': 6721,
 'car': 7034,
 'is': 22864,
 'waking': 44050,
 'up': 43065,
 'in': 21747,
 'bathtub': 4900,
 'ice': 21330,
 'without': 4491

For the three techniques in problem (2) above, give an example where each would be useful.

Word count vector - can help identify a subject of a text and a hidden subjet within text.  Can also be used for sentiment analysis - THere are references out there that use sentiment analysis to their benefit.  One example would be an airline company that reads comments from Twitter.  If the airline company identifies certain words that have a negative sentiment, they can look out for these words and make marketing adjustments as necessary to help with customer satisfaction.

Part of speech tag vector - Tagging parts of speech is important in that it allows for the context of a word to be identified.  For example, if we continue with the twitter comment airline example, the word "late" without any context would probably be viewed as negative when it applies to an airline company.  For example, if a comment was " #airlinecompany flight 1234 is running late as usual!" then the word "late" in this context has a negative sentiment.  However, if another comment was "#airlinecompany made sure that I made my flight even though I got to the airport really late.", then that can be viewed positively.

Term frequency - inverse document frequency - One of the most important uses of term frequency - inverse document frequency is found in something we use almost every day, Google search.  When we search for a term in Google, the words we search are compared with all the text that Google is able to search in its universe.  Those terms are compared with the texts Google searches upon, and by using the Term Frequency - Inverse Document Frequency, it is able to return the most relevant documents (websites) to our terms.